In [117]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import mean_squared_log_error
import string
from nltk.corpus import stopwords
import time
os.chdir(r"...\Amazon Project")

In [125]:
data = pd.read_csv('AMAZON_comments1.csv',dtype=str)
df = data.copy()
df = df.dropna(axis=0)
text = df['Text']

In [121]:
def stop_punch(text):
    # Clean out all the stopwords
    stext = [word for word in text.lower().split() if word not in set(stopwords.words('english'))]
    ftext = ' '.join(stext)
    
    # Clean out all the punctuations
    stext = [word for word in ftext if word not in set(string.punctuation)]
    ftext = ''.join(stext)
    return ftext

In [128]:
# Clean some of the repetitive words.
text = text.apply(lambda x: x.replace('Format',""))
text = text.apply(lambda x: x.replace('Wireless Phone Accessory',""))
text = text.apply(lambda x: x.replace('Color',""))
text = text.apply(lambda x: x.replace('Verified Purchase',""))
text = text.apply(lambda x: x.replace('people found this helpful',""))

In [129]:
t0 = time.time()
text = text.apply(stop_punch)
print("Completed within %0.1f seconds." % (time.time() - t0))

Completed within 902.2 seconds.


In [84]:
text[0]

'You wont believe what you get for 50 bucks\r\r\nByLuisP8on June 18 2016\r\r\nColor Black\r\r\nVerified Purchase\r\r\nI must say and admit I was a little concerned about this choice after I bought it but I said Well what the heck lets give it a try It was for my mom anyway but on paper it looked like a nice budget phone and also I had extra concerns cause Im not from usa so retuning this was out of the table I had to accept whatever came on the box Fortunately I was wrong The product came in perfect condition and brand new of course no problems with packaging or shipping And after first hands on the phone you wont believe you pay 60 bucks after taxes and handling for this phone I MEAN IT This guys BLU Inc  are doing an awesome job with budget phones This phone is really awesome Of course it doesnt look fancy or premium because its obviously plastic but you fell like you have a 500 phone in your hands\r\r\nSo if you are looking for a budget phone DO NOT hesitate of this one you wont be 

In [135]:
text[0]

'believe get 50 bucks byluisp8on june 18 2016  black must say admit little concerned choice bought said well heck lets give try mom anyway paper looked like nice budget phone also extra concerns cause im usa retuning table accept whatever came box fortunately wrong product came perfect condition brand new course problems packaging shipping first hands phone believe pay 60 bucks after taxes handling phone mean it guys blu inc  awesome job budget phones phone really awesome course look fancy premium obviously plastic fell like 500 phone hands looking budget phone hesitate one sorry 410 helpful helpful 5 comments report abuse'

# Vectorization

### CountVectorizer

In [139]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
count_vec = CountVectorizer(ngram_range=(1, 3),
                            max_df=0.50,
                            analyzer='word',
                           token_pattern = r'\w{2,}'
                           )
feature_train_counts = count_vec.fit(text)
bag_of_words = feature_train_counts.transform(text)

from sklearn.model_selection import train_test_split
X = bag_of_words 
y = df['Stars'].astype(float)
feature_train, feature_test, label_train, label_test = train_test_split(X, y, test_size=0.33, shuffle=True)

In [140]:
bag_of_words

<50460x1916715 sparse matrix of type '<class 'numpy.int64'>'
	with 5747177 stored elements in Compressed Sparse Row format>

# Model Building

## SGDClassifier

In [146]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, log_loss

clf = SGDClassifier(loss='log').fit(feature_train, label_train)
preds_sgd = clf.predict(feature_test)
preds_proba = clf.predict_proba(feature_test)

print(confusion_matrix(label_test, preds_sgd))
print(classification_report(label_test, preds_sgd))
print('Valid RMSLE: {:.4f}'.format(np.sqrt(mean_squared_log_error(label_test, preds_sgd))))
print("F1_score : {:.2%} ".format(f1_score(label_test, preds_sgd, average='micro')))  # 'samples', 'weighted', 'macro', 'micro', 'binary'
print("Log_loss : {:.4f} ".format(log_loss(label_test, preds_proba)))


"""
Valid RMSLE: 0.2126
F1_score : 80.53% 
Log_loss : 1.3345 

Valid RMSLE: 0.2194
F1_score : 80.47% 
Log_loss : 0.7738 

Valid RMSLE: 0.2286
F1_score : 80.48% 
Log_loss : 0.7692  # trimmed a little
"""

c:\users\dogus\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[[3292  101   64   53  122]
 [ 452  520   96   57   90]
 [ 258  103  762  211  207]
 [  88   32  106 1703  730]
 [ 104   22   40  326 7113]]
             precision    recall  f1-score   support

        1.0       0.78      0.91      0.84      3632
        2.0       0.67      0.43      0.52      1215
        3.0       0.71      0.49      0.58      1541
        4.0       0.72      0.64      0.68      2659
        5.0       0.86      0.94      0.90      7605

avg / total       0.79      0.80      0.79     16652

Valid RMSLE: 0.2221
F1_score : 80.41% 
Log_loss : 0.7619 


'\nValid RMSLE: 0.2126\nF1_score : 80.53% \nLog_loss : 1.3345 \n\nValid RMSLE: 0.2194\nF1_score : 80.47% \nLog_loss : 0.7738 \n\nValid RMSLE: 0.2286\nF1_score : 80.48% \nLog_loss : 0.7692  # trimmed a little\n'

## LogisticRegression

In [142]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=10).fit(feature_train, label_train)
preds_lr = clf.predict(feature_test)
preds_proba = clf.predict_proba(feature_test)

print(confusion_matrix(label_test, preds_lr))
print(classification_report(label_test, preds_lr))
print('Valid RMSLE: {:.4f}'.format(np.sqrt(mean_squared_log_error(label_test, preds_lr))))
print("F1_score : {:.2%} ".format(f1_score(label_test, preds_lr, average='micro')))  # 'samples', 'weighted', 'macro', 'micro', 'binary'
print("Log_loss : {:.4f} ".format(log_loss(label_test, preds_proba)))

[[3271  107   64   56  134]
 [ 420  529   99   61  106]
 [ 258   87  786  185  225]
 [  83   37  107 1649  783]
 [ 104   17   46  260 7178]]
             precision    recall  f1-score   support

        1.0       0.79      0.90      0.84      3632
        2.0       0.68      0.44      0.53      1215
        3.0       0.71      0.51      0.59      1541
        4.0       0.75      0.62      0.68      2659
        5.0       0.85      0.94      0.90      7605

avg / total       0.80      0.81      0.79     16652

Valid RMSLE: 0.2245
F1_score : 80.55% 
Log_loss : 0.8122 


In [ ]:
"""
Valid RMSLE: 0.2063
F1_score : 81.20% 
Log_loss : 0.8746 

Valid RMSLE: 0.2207
F1_score : 80.30% 
Log_loss : 0.8388 

Valid RMSLE: 0.2270
F1_score : 80.54% 
Log_loss : 0.8218 # trimmed a little
"""